In [1]:
import pandas as pd
import json
import regex
import os
def preprocess_logic(input_paths):
    os.makedirs('./new_data', exist_ok=True)
    label_map = {
        "faulty generalization" : "Faulty Generalization",
        "appeal to emotion": "Appeal to Emotion",
        "ad hominem": "Ad Hominem",
        "fallacy of relevance": "Red Herring",
        "false causality" : "False Causality (Post Hoc Fallacy)",
        "circular reasoning" : "Circular Reasoning",
        "ad populum": "Ad Populum",
        "fallacy of logic": "Fallacy of Converse (Affirming the Consequent)",
        "false dilemma": "False Dilemma",
        "equivocation": "Equivocation",
        "fallacy of extension": "Straw Man",
        "fallacy of credibility" : "Doubt Credibility",
        "intentional": "Intentional (Intentionally Wrong Argument)"
    }
    for input_path in input_paths:
        split = input_path.split(".")[1].split("_")[-1]
        df = pd.read_csv(input_path, on_bad_lines='skip')
        df = df.drop(df[(df['source_article'].isna()) | (df['updated_label'].isna())].index)
        df = df.reset_index(drop=True)
        df['id'] = df.index
        df = df[['id','source_article', 'updated_label']]
        df['text'] = df['source_article'].apply(lambda x: " ".join(x.strip().replace("\u2019", "'").encode('ascii', errors='ignore').strip().decode('ascii').replace('"""', '').split()))
        df['label'] = df['updated_label'].apply(lambda x: [label_map[x]])
        
        df = df.drop(columns=['source_article', 'updated_label'])
        print(f"{split}: A total of {len(df.index)} examples.")
        print(df['label'].value_counts())
        print()
        js = df.to_json(orient="records", indent=4)#lines=True
        
        with open(f'./new_data/{split}.json', 'w') as f:
            f.write(js)
            f.close()
    return
preprocess_logic(['./edu_train.csv','./edu_dev.csv','./edu_test.csv'])


train: A total of 1849 examples.
label
[Faulty Generalization]                             319
[Ad Hominem]                                        225
[False Causality (Post Hoc Fallacy)]                174
[Ad Populum]                                        158
[Circular Reasoning]                                134
[Appeal to Emotion]                                 130
[Fallacy of Converse (Affirming the Consequent)]    121
[Red Herring]                                       114
[Intentional (Intentionally Wrong Argument)]        112
[False Dilemma]                                     110
[Doubt Credibility]                                 107
[Straw Man]                                         106
[Equivocation]                                       39
Name: count, dtype: int64

dev: A total of 300 examples.
label
[Faulty Generalization]                             61
[Ad Populum]                                        44
[Ad Hominem]                                        36
[Fals

In [2]:
label_map = {
        "faulty generalization" : "Faulty Generalization",
        "appeal to emotion": "Appeal to Emotion",
        "ad hominem": "Ad Hominem",
        "fallacy of relevance": "Red Herring",
        "false causality" : "False Causality (Post Hoc Fallacy)",
        "circular reasoning" : "Circular Reasoning",
        "ad populum": "Ad Populum",
        "fallacy of logic": "Fallacy of Converse (Affirming the Consequent)",
        "false dilemma": "False Dilemma (Excluding viable alternatives)",
        "equivocation": "Equivocation (Use of ambiguous language)",
        "fallacy of extension": "Fallacy of Extension (Exaggeration)",
        "fallacy of credibility" : "Attack Credibility (Doubt)",
        "intentional": "Intentional (Intentionally wrong argument)"
    }
for k, v in label_map.items():
    print(v)

Faulty Generalization
Appeal to Emotion
Ad Hominem
Red Herring
False Causality (Post Hoc Fallacy)
Circular Reasoning
Ad Populum
Fallacy of Converse (Affirming the Consequent)
False Dilemma (Excluding viable alternatives)
Equivocation (Use of ambiguous language)
Fallacy of Extension (Exaggeration)
Attack Credibility (Doubt)
Intentional (Intentionally wrong argument)


In [9]:
import pandas as pd
mapping = pd.read_csv("./mappings.csv")
mapping[['Original Name', 'Understandable Name', 'Description']]# 'Description'
for fal in mapping['Original Name'].to_list():
    print(f"{fal}: {mapping.loc[mapping['Original Name']==fal,'Logical Form' ].values}")

faulty generalization: ['Sample S is taken from population P. Sample S is a very small part of population P. Conclusion C is drawn from sample S and applied to population P.']
false causality: ['A occurred, then B occurred.\nTherefore, A caused B.\n ']
circular reasoning: ['X is true because of Y.\n\nY is true because of X.']
ad populum: ['A lot of people believe X.Therefore, X must be true.']
ad hominem: ['Person 1 is claiming Y.\n\nPerson 1 is a moron.\n\nTherefore, Y is not true.']
fallacy of logic: ['If A is true, then B is true. B is true. Therefore, A is true.']
appeal to emotion: ['Claim X is made without evidence. In place of evidence, emotion is used to convince the interlocutor that X is true.']
false dilemma: ['Either X or Y is true.']
equivocation: ['Term X is used to mean Y in the premise.\nTerm X is used to mean Z in the conclusion.']
fallacy of extension: ['Person 1 makes claim Y. Person 2 restates person 1’s claim (in a distorted way). Person 2 attacks the distorted ver

In [17]:
print(mapping.loc[mapping['Original Name'] == 'fallacy of logic', 'Logical Form'].values)

['If A is true, then B is true. B is true. Therefore, A is true.']
